In [2]:
! pip install --user shapely geopandas pyproj xxhash

     |█▋                              | 92 kB 2.5 MB/s eta 0:00:01

     |███████▏                        | 409 kB 2.3 MB/s eta 0:00:01

     |██████████████▋                 | 829 kB 2.3 MB/s eta 0:00:01

     |███████████████████████         | 1.3 MB 2.3 MB/s eta 0:00:01

     |███████████████████████████████▉| 1.8 MB 2.3 MB/s eta 0:00:01

...

     |██████████████████████████████▍ | 13.9 MB 21.3 MB/s eta 0:00:01

     |████████████████████████████████| 14.7 MB 21.3 MB/s eta 0:00:01

     |████████████████████████████████| 14.7 MB 231 kB/s 


     |████████████████████████████████| 82 kB 1.2 MB/s 


You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [3]:
import shapely
import geopandas as gpd
import pandas as pd
import shapely.geometry.polygon as ply
import xxhash

def add_geo(r, geo):
    r['geometry'] = geo
    return r

def split_geos(r):
    if isinstance(r.geometry, ply.Polygon):
        return pd.DataFrame([r])
    df = pd.DataFrame([add_geo(r, geo) for geo in r.geometry])
    return df

def bufferz(c, rad, d):
    r = c.combine_first(d)
    r['geometry'] = c.geometry.buffer(rad)

    return r, (c.osm_id, d.distcode, d.distname, d.statename, c.geometry, rad/1000)

def hsh(n):
    return xxhash.xxh32(n).hexdigest()

def make_clusters(cities, dists, buffer_margin):
    dat = [(c, d) for _, d in dists.iterrows()
         for _, c in cities[cities.within(d.geometry)].iterrows() ]   

    dat = [(c, d.geometry.exterior.distance(c.geometry) - buffer_margin, d) for c, d in dat]
    dat = [bufferz(*t) for t in dat]
    rows = [r for r, _ in dat]
    
    cities_df = pd.DataFrame([t for _, t in dat], columns=['id', 'distcode', 'distname', 'state', 'geometry', 'rad'])
    cities_df = gpd.GeoDataFrame(cities_df, geometry='geometry', crs=3857).to_crs(4326)
    cities_df['lng'] = cities_df.geometry.map(lambda g: g.coords[0][0])
    cities_df['lat'] = cities_df.geometry.map(lambda g: g.coords[0][1])
    cities_df = pd.DataFrame(cities_df)
    cities_df = cities_df.drop(columns=['geometry'])
    cities_df['disthash'] = cities_df.distname.map(hsh)

    # fix one city issue!
    cities_df.loc[cities_df.distname == 'Balrampur', 'distcode'] = '182'

    gdf = gpd.GeoDataFrame(pd.DataFrame(rows), geometry='geometry', crs=3857) \
             .to_crs(4326) \
             .reset_index(drop=True)
    return gdf, cities_df

In [114]:
# Uttar Pradesh: 
# Jharkhand: Ranchi
# Chhattisgarh: 
# Odisha: Khordha

In [4]:
cities = gpd.read_file('India_MNM/Geography/Cities_India/cities_towns_suburbs.shp')
districts = gpd.read_file('India_MNM/Geography/Districts+Demographics_India/Demographics_of_India.shp')

# , 'Jharkhand', 'Chhatisgarh', 'Orissa', 'Uttar Pradesh'
# states = ['Jharkhand']
states = ['Jharkhand', 'Chhatisgarh', 'Uttar Pradesh']
capitols = ['Lucknow', 'Ranchi', 'Raipur', 'Khordha']

# Split geos to make one row per polygon
dists = pd.concat([split_geos(r) for _,r in districts.iterrows()]).reset_index(drop=True)
dists = gpd.GeoDataFrame(dists, geometry='geometry', crs=4326)
dists = dists[dists.statename.isin(states)]
dists = dists[~dists.distname.isin(capitols)]

# Project to meter distance projection
dists = dists.to_crs(3857)
cities = cities.to_crs(3857)

In [5]:
dd, df = make_clusters(cities, dists, 2000)

In [ ]:


dists[dists.tot_p < 500000]

In [6]:
df.to_csv('output/cities.csv', index=False)

In [117]:
dd.to_file('output/clusters.shp')

In [ ]:
dd

In [ ]:
df.distname.unique().shape

In [ ]:
dd.columns

In [ ]:
df[df.distcode == '#N/A']